<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/Test_SMILES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget http://file.weasoft.com/main.pb -nv
!wget http://file.weasoft.com/80k.csv -nv
!wget http://file.weasoft.com/swin_transform_focalloss.pth -nv
!wget http://file.weasoft.com/80k.zip
!unzip -q 80k.zip
!pip3 install torch_xla
!pip3 install deepsmiles yacs tqdm
!git clone https://github.com/suanfaxiaohuo/SwinOCSR.git
!pip install focal_loss_torch wandb

2023-11-12 07:44:05 URL:http://file.weasoft.com/main.pb [510999187/510999187] -> "main.pb" [1]
2023-11-12 07:44:08 URL:http://file.weasoft.com/80k.csv [172176229/172176229] -> "80k.csv" [1]
2023-11-12 07:44:37 URL:http://file.weasoft.com/swin_transform_focalloss.pth [2555716718/2555716718] -> "swin_transform_focalloss.pth" [1]
--2023-11-12 07:44:37--  http://file.weasoft.com/80k.zip
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3356218517 (3.1G) [application/zip]
Saving to: ‘80k.zip’

80k.zip             100%[===================>]   3.12G  83.8MB/s    in 40s     

2023-11-12 07:45:17 (81.0 MB/s) - ‘80k.zip’ saved [3356218517/3356218517]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cloning into 'SwinOCSR'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 453 (delta 104), reused 77 (delta 42), pack-reused 279
Receiving objects: 100% (453/453), 220.65 MiB | 49.92 MiB/s, done.
Resolving deltas: 100% (201/201), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.3 MB/s eta 0:00:00


In [1]:
import torch, torchvision
import os
import numpy as np
import pylab
import pandas
import sys
from PIL import Image

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    HOME_DIR = "/content"
else:
    HOME_DIR = "/arc/burst/st-dushan20-1/"

In [3]:

# From the SwinOCSR
import sys
sys.path.append("/content/SwinOCSR/model/Swin-transformer-focalloss")
sys.path.append("./SwinOCSR/model/")

import sys
import os
import argparse
from tqdm import tqdm
import deepsmiles
from typing import Any, cast, Callable, List, Tuple, Union
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms

from pre_transformer import Transformer

In [4]:
# os.environ['PJRT_DEVICE'] = 'TPU' #That is why
import torch_xla
import torch_xla.core.xla_model as xm
# import torch_xla.runtime as xr
# device = xm.xla_device()
try:
  5/0
  import torch_xla
  import torch_xla.core.xla_model as xmc
  device = xm.xla_device()
except:
  if torch.cuda.is_available():
      device = torch.device('cuda:0')
  else:
      device = torch.device('cpu')

In [5]:
class FocalLossModelInference:
    """
    Inference Class
    """
    def __init__(self):
        # Load dictionary that maps tokens to integers
        word_map_path = './SwinOCSR/Data/500wan/500wan_shuffle_DeepSMILES_word_map'
        self.word_map = torch.load(word_map_path)
        self.inv_word_map = {v: k for k, v in self.word_map.items()}

        # Define device, load models and weights
        self.dev = "cuda" if torch.cuda.is_available() else "cpu"
        # self.args, config = self.get_inference_config()
        # self.encoder = build_model(config, tag=False)
        self.decoder = self.build_decoder()
        self.load_checkpoint("./swin_transform_focalloss.pth")
        self.decoder = self.decoder.to(self.dev).eval()
        # self.encoder = self.encoder.to(self.dev).eval()

    def load_checkpoint(self, checkpoint_path):
        """
        Load checkpoint and update encoder and decoder accordingly

        Args:
            checkpoint_path (str): path of checkpoint file
        """
        print(f"=====> {checkpoint_path} <=====")
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        # encoder_msg = self.encoder.load_state_dict(checkpoint['encoder'],
        #                                            strict=False)
        decoder_msg = self.decoder.load_state_dict(checkpoint['decoder'],
                                                   strict=False)
        # print(f"Encoder: {encoder_msg}")
        print(f"Decoder: {decoder_msg}")
        del checkpoint
        torch.cuda.empty_cache()

    def build_decoder(self):
        """
        This method builds the Transformer decoder and returns it
        """
        self.decoder_dim = 256  # dimension of decoder RNN
        self.ff_dim = 2048
        self.num_head = 8
        self.dropout = 0.1
        self.encoder_num_layer = 6
        self.decoder_num_layer = 6
        self.max_len = 277
        self.decoder_lr = 5e-4
        self.best_acc = 0.
        return Transformer(dim=self.decoder_dim,
                           ff_dim=self.ff_dim,
                           num_head=self.num_head,
                           encoder_num_layer=self.encoder_num_layer,
                           decoder_num_layer=self.decoder_num_layer,
                           vocab_size=len(self.word_map),
                           max_len=self.max_len,
                           drop_rate=self.dropout,
                           tag=False)
transformer_ = FocalLossModelInference()
transformer = transformer_.build_decoder().decoder

converter = deepsmiles.Converter(rings=True, branches=True)


def str_to_vector(s: str)->list:
  return [transformer_.word_map[i] for i in converter.encode(s)]


import pandas as pd
csv = pd.read_csv("./80k.csv")
cids = csv["cid"]
csv.columns


=====> ./swin_transform_focalloss.pth <=====
Decoder: <All keys matched successfully>


Index(['cid', 'cmpdname', 'cmpdsynonym', 'mw', 'mf', 'polararea', 'complexity',
       'xlogp', 'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds', 'inchi',
       'isosmiles', 'canonicalsmiles', 'inchikey', 'iupacname', 'exactmass',
       'monoisotopicmass', 'charge', 'covalentunitcnt', 'isotopeatomcnt',
       'totalatomstereocnt', 'definedatomstereocnt', 'undefinedatomstereocnt',
       'totalbondstereocnt', 'definedbondstereocnt', 'undefinedbondstereocnt',
       'pclidcnt', 'gpidcnt', 'gpfamilycnt', 'neighbortype', 'meshheadings',
       'annothits', 'annothitcnt', 'aids', 'cidcdate', 'sidsrcname', 'depcatg',
       'annotation'],
      dtype='object')

In [6]:
Ys = {}
invalid_cids = []
for i in cids.values:
    tmp = str_to_vector(csv[csv["cid"] == i]["canonicalsmiles"].values[0])
    if not tmp == None:
      Ys[i] = tmp
    else:
      invalid_cids.append(i)

print("OOHH")

OOHH


In [7]:
from focal_loss.focal_loss import FocalLoss
m = torch.nn.Softmax(dim=-1)
lf = FocalLoss(gamma=2, ignore_index=0)#torch.nn.CrossEntropyLoss(label_smoothing=0.1, reduction="none")
def loss_fn(pred, truth):
  pred = m(pred)
  l = lf(pred, truth)
  return l


def mask_acc(pred, truth):
    pred = torch.argmax(pred, -1)
    mask = truth != 0
    match_case = truth == pred
    return torch.sum(mask*match_case)/torch.sum(mask)



import pickle

BATCH_SIZE = 4
files = os.listdir(f"./rendered/")
files = [i for i in files if len(i)<=40]
import multiprocessing, threading
#https://stackoverflow.com/questions/48822463/how-to-use-pytorch-multiprocessing
from torch.multiprocessing import Process, Queue, Pool
import time
buffer = Queue(maxsize=10) #need maxsize=10, otherwise put will also block
start_index = 0
import cv2
def process_single(arg):
    # print(arg)
    _, start_index = arg
    index = start_index + _
    try:
      id = int(files[index].split("_")[0])
    except:
      return
    img = np.array(Image.open(f"{HOME_DIR}/rendered/{files[index]}")) #cannot read image? quota reached?
    return img, [77] + Ys[id], Ys[id] + [78]

#pool = Pool()


#read all images
images_warehouse = []
index = 0
import time
t0 = time.time()
import os, psutil
process = psutil.Process()


# reset

In [8]:
import gc

In [29]:

# https://stackoverflow.com/questions/70508960/how-to-free-gpu-memory-in-pytorch

try:
  model.cpu()
  gen.cpu()
  del model, gen
  gc.collect()
  torch.cuda.empty_cache()
except:
  pass
example_in = Image.open("./rendered/6912034_0.jpg")
example_out = csv[csv["cid"]==6912034]["canonicalsmiles"].values[0]

# In[15]:


example_out = str_to_vector(example_out)


torch.permute(torch.tensor(np.expand_dims(example_in, 0).astype("float32")), (0,3,1,2)).size()


t = torch.randn(1,64,400,400)
torch.flatten(t)
torch.flatten(t, start_dim=2,end_dim=3).shape


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

import numpy as np
def positional_encoding_1d(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model
    )

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return pos_encoding


def positional_encoding_2d(row, col, d_model):
    assert d_model % 2 == 0
    row_pos = np.repeat(np.arange(row), col)[:, np.newaxis]
    col_pos = np.repeat(np.expand_dims(np.arange(col), 0), row, axis=0).reshape(-1, 1)

    angle_rads_row = get_angles(
        row_pos, np.arange(d_model // 2)[np.newaxis, :], d_model // 2
    )
    angle_rads_col = get_angles(
        col_pos, np.arange(d_model // 2)[np.newaxis, :], d_model // 2
    )

    angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
    angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
    angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
    angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
    pos_encoding = np.concatenate([angle_rads_row, angle_rads_col], axis=1)[
        np.newaxis, ...
    ]
    return pos_encoding

eff = torchvision.models.efficientnet_v2_l(weights='DEFAULT')
mynet = eff.features
class ImageEncoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.eff = mynet.to(device)
    self.projection = torch.nn.Linear(1280,256).to(device)
  def forward(self, images):
    features = self.eff(images)
    features = torch.flatten(features, start_dim=2, end_dim=3)
    features = torch.permute(features, (0, 2, 1))
    return self.projection(features)



encoder = ImageEncoder()

for p in transformer.parameters():
  p.requires_grad = False


# In[45]:
# from another paper
def pad_pack(sequences):
    maxlen = max(map(len, sequences))
    batch = torch.LongTensor(len(sequences),maxlen).fill_(0)
    for i,x in enumerate(sequences):
        batch[i,:len(x)] = torch.LongTensor(x)
    return batch, maxlen

# https://github.com/suanfaxiaohuo/SwinOCSR/blob/main/model/Swin-transformer-focalloss/pre_transformer.py#L95
def triangle_mask(size):
    mask = 1- np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    mask = torch.autograd.Variable(torch.from_numpy(mask))
    return mask


class Image2SMILES(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, image, text_in, xmask):
    #padded_text, maxlen = pad_pack(text_in)
    #padded_text = padded_text.to(device)
    # image = torch.permute(torch.tensor(np.array(image)), (0,3,1,2))
    image_feature = self.encoder(image)
    out = self.decoder(text_in, image_feature, x_mask=xmask)#triangle_mask(maxlen).to(device))
    return out

# In[46]:

def softmax(x):
        t = np.exp(x)
        return t/np.sum(t)

softmax_1 = torch.nn.Softmax(dim=-1)
torch_softmax = torch.nn.Softmax()
class SMILESGenerator(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, image, text_in_, max_len, beam=1): #just changed beam=1 and it runs so beam cannot be random number???  yeah it has to been factor of 676, which is 2 2 13 13 but original paper did not use beam search
    image_feature = self.encoder(image)
    top_n = list([list(i) for i in torch.tensor(text_in_).repeat(beam,1).detach().cpu().numpy()])
    # top_n_conf = torch.tensor([1.] * beam).to(device) #we do not need this, we just need to track current best

    for i in range(max_len):
      if beam == 1:
        padded_text, l = pad_pack(top_n)
        padded_text = padded_text.to(device)
        out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
        out = out[0,-1,:]
        next = torch.argmax(out)
        top_n[0] += [next.detach().cpu().item()]
        if next == 78:
          return top_n
        continue
      # top_n_conf = torch_softmax(top_n_conf)
      # print(top_n)
      padded_text, l = pad_pack(top_n)
      padded_text = padded_text.to(device)
      out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
      out = softmax_1(out[:,-1,:])
      next = torch.topk(torch.flatten(out), beam)
      indices2d = np.array(np.unravel_index(next.indices.cpu().numpy(), out.shape)).T
      new_top_n = []
      count = 0
      for j in indices2d:
        beam_number = j[0]
        char = j[1]
        # if 78 in top_n[beam_number]:
        if char == 78:
          count += 1
          new_top_n.append(top_n[beam_number])
        else:
          new_top_n.append(top_n[beam_number] + [char])
      if count == beam:
        return new_top_n
      top_n = new_top_n

    return top_n


model = Image2SMILES(encoder, transformer)
model.load_state_dict(torch.load("./main.pb", map_location=torch.device("cpu")))
gen = SMILESGenerator(model.encoder, model.decoder)
for param in model.parameters():
    param.requires_grad = False

for param in gen.parameters():
    param.requires_grad = False

model = model.to(device)
gen = gen.to(device)


def softmax(x):
  t = np.exp(x)
  return t/np.sum(t)


In [10]:
next = torch.topk(torch.flatten(torch.tensor([[0,1,3],[100,10000,200000000]])), 1)
np.array(np.unravel_index(next.indices.cpu().numpy(), (2, 3))).T

array([[1, 2]])

In [11]:
torch.nn.Softmax(dim=-1)(torch.tensor([[0.1,0.2,3000],[0.2, 0.6, 0.9],[0.9, 0.7, 0.1]]))

tensor([[0.0000, 0.0000, 1.0000],
        [0.2219, 0.3311, 0.4469],
        [0.4409, 0.3610, 0.1981]])

In [12]:
torch.topk(torch.tensor([[0.1,0.2,0.3],[0.2, 0.6, 0.9],[0.9, 0.7, 0.1]]), 1)

torch.return_types.topk(
values=tensor([[0.3000],
        [0.9000],
        [0.9000]]),
indices=tensor([[2],
        [2],
        [0]]))

In [13]:
# prompt: transpose torch.broadcast_to(torch.tensor([0.9, 0.1, 0.2]), (10, 3))

torch.broadcast_to(torch.tensor([0.9, 0.1, 0.2]), (10, 3)).T * torch.rand((3, 10))


tensor([[0.6338, 0.6082, 0.0319, 0.6699, 0.8123, 0.8216, 0.8957, 0.6078, 0.7754,
         0.5828],
        [0.0215, 0.0931, 0.0400, 0.0267, 0.0131, 0.0221, 0.0436, 0.0827, 0.0931,
         0.0549],
        [0.0514, 0.0780, 0.1914, 0.0499, 0.0395, 0.1014, 0.1946, 0.0749, 0.0770,
         0.0871]])

In [14]:
reversed_word_map = {}
for i in transformer_.word_map.keys():
 reversed_word_map[transformer_.word_map[i]] = i

# run

In [30]:
import wandb
print("Started")

optimizer = torch.optim.AdamW(
   model.parameters(),
   lr=0.0003)


scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99987)


loss_list = []
model.to(device)


def getitem(i):
 imgs = []
 ti = []
 to = []
 for i in range(i, i+BATCH_SIZE):
    index = i
    try:
      id = int(files[index].split("_")[0])
    except:
      return
    try:
     img = np.array(Image.open(f"./rendered/{files[index]}")) #cannot read image? quota reached?
     imgs.append(img)
     ti.append([77] + Ys[id])
     to.append(Ys[id] + [78])
    except:
     pass
 return torch.tensor(np.array(imgs), dtype=torch.float32), ti, to

Test = True
if Test:
  model.train(False)
  gen.train(False)
  running_loss = 0
  last_loss = 0
  tf_count = 0
  bs_count = 0
  for i in range(0, 100):
    start_index = i * BATCH_SIZE
    Xs_img = []
    Xs_text = []
    y = []

    image, text_in, text_out = getitem(start_index)
    image = image.to(device)
    text_out = pad_pack(text_out)[0].to(device)
    padded_x = pad_pack(text_in)
    xmask = triangle_mask(padded_x[1]).to(device)
    image = torch.permute(image, (0, 3, 1, 2))
    text_in = padded_x[0].to(device)

    outputs = model(image, text_in, xmask)[0].cpu().detach().numpy()
    tf = "".join([reversed_word_map[np.argmax(i)] for i in outputs]).replace("<end>","").replace("<start>","").replace("<pad>","")
    bs = "".join([reversed_word_map[i] for i in gen(image, [[77]], len(text_in[0]), beam=1)[0]]).replace("<end>","").replace("<start>","").replace("<pad>","")
    correct = "".join([reversed_word_map[i] for i in text_out[0].detach().cpu().numpy()]).replace("<end>","").replace("<start>","").replace("<pad>","")
    if tf == correct:
      tf_count += 1
    if bs == correct:
      bs_count += 1
    print(tf)
    print(bs)
    print(correct) #print le cai fa xian you <end>
    # print("Teacher forcing output:\t\t", "".join([reversed_word_map[np.argmax(i)] for i in outputs]))
    # print("Token by Token Output:\t\t", "".join([reversed_word_map[i] for i in gen(image, [[77]], len(text_in[0]))[0]]))
    # print("Correct output:\t\t\t", "".join([reversed_word_map[i] for i in text_out[0].detach().cpu().numpy()]))
    print()
else:
  for epoch in range(30):
    image = Xs_img.to(device) #mutli process cannot use cuda so moved here
    image = torch.permute(image, (0, 3, 1, 2))
    text_out = pad_pack(text_out)[0].to(device)
    padded_x = pad_pack(text_in)

    xmask = triangle_mask(padded_x[1]).to(device)
    text_in = padded_x[0].to(device)
    optimizer.zero_grad()
    outputs = model(image, text_in, xmask)
    loss = loss_fn(outputs, text_out)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()

    if i%100==99:
        wandb.log({"loss": running_loss/100, "acc":mask_acc(outputs.detach(), text_out), "lr": optimizer.param_groups[0]['lr']})
        running_loss = 0.
    if i%10 == 9:
        scheduler.step()

    if i == 10:
        model.train(False)
        for ii in range(0,10):
            Xs_img, text_in, text_out = getitem(ii * BATCH_SIZE)

            image = Xs_img.to(device) #mutli process cannot use cuda so moved here
            image = torch.permute(image, (0, 3, 1, 2))
            text_out = pad_pack(text_out)[0].to(device)
            padded_x = pad_pack(text_in)
            xmask = triangle_mask(padded_x[1]).to(device)
            text_in = padded_x[0].to(device)
            outputs = model(image, text_in, xmask)
            loss = loss_fn(outputs, text_out)
            wandb.log({"val_loss": loss, "val_acc":mask_acc(outputs.detach(), text_out)})
        model.train(True)

Started
CC=O)CC=O)SCCNC=O)C
C=CC=CC=C6CN)))NC#N
CC=O)CC=O)SCCNC=O)C

CCC=CC=CC=C6)O))))OC6
CCC=CC=C=C6C6)))))OC6
CCC=CC=CC=C6)O))))OC6

CCC)C)CCC)C)O)
CCCC))CC=CC=C6)))O)O
CCC)C)CCC)C)O

CCNC=O)C=CC=CC=C6))O
CC=C=O)C=CC=C=O)N))O
CCNC=O)C=CC=CC=C6))O

CNCCO)))C=O)
CCCCCC#N+])NC
CNCCO)))C=O

CC=NNC=S)NC6=O))N
CCCCCNC=O)N6=O)C6)O
CC=NNC=S)NC6=O))N

CS=O)CCCC=O)[O-]))[NH3+]
CC=O)CCC=C=O))))C
CS=O)CCCC=O)[O-]))[NH3+]

CCC)CC=O)C=CC=CC=C6)))))))Br
CCCCC=O)O5)Cl))O
CCC)CC=O)C=CC=CC=C6)))))))Br

CC=O)CCCC4
CC=O)=CC=C6S=Cl
CC=O)CCCC4

C=CC=NC=C6Br))))CF)F)F))
CCNCC=CC=C6C=O)C9=NO
C=CC=NC=C6Br))))CF)F)F

CCNC=NC=CC=CC=C6N9C%10
CCCCCCC=C6CC=NO)N
CCNC=NC=CC=CC=C6N9C%13

CS=O)=O)C=CC=CC=C6))C=O
CCCS=O)C=O)O)CC=C=O
CS=O)=O)C=CC=CC=C6))C=O

C=CC=CC=C6F))F))F)))N
C=CC=NON=C6F)F)F
C=CC=CC=C6F))F))F)))N



KeyboardInterrupt: ignored